Loading Quotes

In [52]:
import pandas as pd

# Load only the Company_Name and Quote_Code of the latest Publication_Date
quotes_df = pd.read_csv('quotes_potential.csv', usecols=['Company_Name', 'Quote_Code', 'Publication_Date'])
quotes_df['Publication_Date'] = pd.to_datetime(quotes_df['Publication_Date'])
latest_date = quotes_df['Publication_Date'].max()
latest_quotes_df = quotes_df[quotes_df['Publication_Date'] == latest_date][['Quote_Code', 'Company_Name']]
print(f"✅ Loaded {len(latest_quotes_df)} quotes from the latest published date: {latest_date.strftime('%Y-%m-%d')}")

✅ Loaded 86 quotes from the latest published date: 2026-02-07


mapping

In [53]:
import requests

headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Origin': 'https://irbe7.com',
    'Referer': 'https://irbe7.com/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'Sec-GPC': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not(A:Brand";v="8", "Chromium";v="144", "Brave";v="144"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

params = {
    'limit': '30',
    'query': '',
    'type': '',
    'exchange': 'TN',
}

response = requests.get('https://data.irbe7.com/api/data/search', params=params, headers=headers)
mapping_data = response.json()
if isinstance(mapping_data, list):
    symbol_to_ticker = {item['symbol']: item['ticker'] for item in mapping_data}
else:
    symbol_to_ticker = {mapping_data['symbol']: mapping_data['ticker']}

print(f"Created mapping for {len(symbol_to_ticker)} symbols")
print("Sample mappings:", dict(list(symbol_to_ticker.items())[:5]))
latest_quotes_df['Quote_Code_mapped'] = latest_quotes_df['Quote_Code'].map(symbol_to_ticker)
# Handle any unmatched symbols (keep original if no mapping found)
unmatched = latest_quotes_df['Quote_Code_mapped'].isna().sum()
if unmatched > 0:
    print(f"⚠️  {unmatched} symbols had no mapping and were set to NaN")

Created mapping for 94 symbols
Sample mappings: {'AETEC': 'AETECH', 'AL': 'AIR LIQUIDE TSIE', 'ALKIM': 'ALKIMIA', 'AB': 'AMEN BANK', 'AMS': 'AMS'}
⚠️  6 symbols had no mapping and were set to NaN


prices

In [54]:
from datetime import datetime, timedelta

def get_timestamps(years=15):
    """Get 'from' and 'to' timestamps for the last N years"""
    to_date = datetime.now()
    from_date = to_date - timedelta(days=years * 365)
    
    # Convert to Unix timestamps (seconds)
    to_timestamp = int(to_date.timestamp())
    from_timestamp = int(from_date.timestamp())
    
    return from_timestamp, to_timestamp

In [ ]:
import pandas as pd
import time

from_ts, to_ts = get_timestamps(years=15)
print(f"Fetching data from {datetime.fromtimestamp(from_ts)} to {datetime.fromtimestamp(to_ts)}")

prices = pd.DataFrame([])
for quote in latest_quotes_df['Quote_Code_mapped']:
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
        'Origin': 'https://irbe7.com',
        'Referer': 'https://irbe7.com/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'Sec-GPC': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not(A:Brand";v="8", "Chromium";v="144", "Brave";v="144"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    params = {
        'symbol': f'{quote}',
        'resolution': '1D',
        'from': f'{from_ts}',
        'to': f'{to_ts}',
        'countback': '365',
    }

    response = requests.get('https://data.irbe7.com/api/data/history', params=params, headers=headers)

    data = response.json()
        
    # Check if data is valid
    if not data or data.get('s') != 'ok' or not data.get('t'):
        print(f"❌ No quote data found for {quote}")
        continue
    
    df = pd.DataFrame({
        'Date': pd.to_datetime(data['t'], unit='s'),
        'Open': data['o'],
        'High': data['h'],
        'Low': data['l'],
        'Close': data['c'],
        'Volume': data['v']
    })
    
    df['Quote_Code'] = latest_quotes_df.loc[latest_quotes_df['Quote_Code_mapped'] == quote, 'Quote_Code'].values[0] 
    prices = pd.concat([prices, df], ignore_index=True)
    
    print(f"✅ {quote}: {len(df)} rows, from {df['Date'].min().date()} to {df['Date'].max().date()}")
    time.sleep(0.3)

# load existing prices if exists
try:
    existing_prices = pd.read_csv('prices.csv')
    prices = pd.concat([existing_prices, prices], ignore_index=True)
    prices.drop_duplicates(subset=['Quote_Code', 'Date'], keep='last', inplace=True)
except FileNotFoundError:
    pass
# save to prices.csv
prices.to_csv('prices.csv', index=False)


Fetching data from 2011-02-11 15:57:04 to 2026-02-07 15:57:04
❌ No quote data found for nan
✅ AETECH: 6799 rows, from 2013-04-08 to 2026-01-23
✅ AIR LIQUIDE TSIE: 14249 rows, from 2010-04-27 to 2026-02-06
✅ AMEN BANK: 28242 rows, from 2010-05-26 to 2026-02-06
✅ AMS: 3211 rows, from 2012-03-02 to 2023-07-05
✅ ATB: 25757 rows, from 2010-04-13 to 2026-02-06
✅ ATL: 25201 rows, from 2010-01-28 to 2026-02-06
✅ ARTES: 26727 rows, from 2010-01-27 to 2026-02-06
✅ ASSAD: 21371 rows, from 2010-04-15 to 2026-02-06
✅ ASSUR MAGHREBIA: 9818 rows, from 2020-12-31 to 2026-02-06
✅ ASTREE: 4447 rows, from 2010-02-19 to 2026-01-26
✅ ATTIJARI BANK: 28692 rows, from 2010-02-02 to 2026-02-06
✅ ATTIJARI LEASING: 20026 rows, from 2010-05-04 to 2026-02-06
✅ BT: 27886 rows, from 2010-02-02 to 2026-02-06
✅ BNA: 28441 rows, from 2010-02-01 to 2026-02-06
✅ BEST LEASE: 6143 rows, from 2013-10-24 to 2026-02-05
✅ BH ASSURANCE: 3014 rows, from 2011-04-13 to 2026-01-28
✅ BH BANK: 22539 rows, from 2010-02-01 to 2026-02-0